## Versioning

TODO:Temporary version is taken from settings.ps1.
    - Since some moment we have two versioning version. Another one is Version.props.
    - For the local repo time-based versioning maybe good to avoid
    - Cleanup local nuget repo in Version.props?
TODO:Other shared projects settings should be also considered
TODO:CI/CD should be responsible for this process

## Local Nuget repoisitory

In [ ]:
function SetupVariable($Name, $Value) {
    New-Item -Path Env: -Name $Name -Value $Value -Force | Out-Null
    [System.Environment]::SetEnvironmentVariable($Name, $Value, [System.EnvironmentVariableTarget]::User)
}

$name = 'XDE Nuget Local'
$folder = Read-Host "XDE local nuget folder"
if (-not(Test-Path $folder)) {
    New-Item -Type Directory -Path $folder | Out-Null
}

SetupVariable -Name 'XDE_NUGET_LOCAL' -Value $folder

$sourceExists = (dotnet nuget list source | sls -Pattern $name).Length -gt 0
if ($sourceExists) {
    dotnet nuget remove source $name
}

dotnet nuget add source $folder --name $name

Package source with Name: XDE Nuget Local removed successfully.
Package source with Name: XDE Nuget Local added successfully.


## Publish packages globally

In [ ]:
$settings = (. ./settings.ps1)
$version = "$($settings.Version).$([datetime]::UtcNow.ToString("yyMMdd.HHmmss"))"
$apiKey = Read-Host "API Key"

dotnet publish ../src `
    --configuration Release `
    /p:PackageVersion=$version `
    /p:Authors=$($settings.Authors)

dotnet nuget push `
    "../src/Xde.Labs/bin/Release/Xde.Labs.$version.nupkg" `
    --api-key $apiKey `
    --source https://api.nuget.org/v3/index.json


Microsoft (R) Build Engine version 17.1.0+ae57d105c for .NET
Copyright (C) Microsoft Corporation. All rights reserved.

  Determining projects to restore...
  Restored C:\Projects\xde\dotnet\src\Xde.Labs\Xde.Labs.csproj (in 76 ms).
  Restored C:\Projects\xde\dotnet\src\Xde.Specs\Xde.Specs.csproj (in 76 ms).
  Xde.Labs -> C:\Projects\xde\dotnet\src\Xde.Labs\bin\Release\net6.0\Xde.Labs.dll
  Successfully created package 'C:\Projects\xde\dotnet\src\Xde.Labs\bin\Release\Xde.Labs.0.0.1-alpha.220412.175031.nupkg'.
  Xde.Labs -> C:\Projects\xde\dotnet\src\Xde.Labs\bin\Release\net6.0\publish\
  Xde.Specs -> C:\Projects\xde\dotnet\src\Xde.Specs\bin\Release\net6.0\Xde.Specs.dll
  Xde.Specs -> C:\Projects\xde\dotnet\src\Xde.Specs\bin\Release\net6.0\publish\
Pushing Xde.Labs.0.0.1-alpha.220412.175031.nupkg to 'https://www.nuget.org/api/v2/package'...
  PUT https://www.nuget.org/api/v2/package/
warn : All published packages should have license information specified. Learn more: https://aka.ms/depre